<h1 align="center"><b>EMS - player Registration</b></h1>

<h5>
    <b> Packages </b>
</h5> 

In [49]:
# Package used to connect to MySQL Databases
import mysql.connector

# XML Creation
import xml.etree.ElementTree as ET
import os

# Data Manipulation Packages
import pandas as pd
import numpy as np
import hashlib
from datetime import datetime

# Package To Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

<h5>
    <b> Import Data From MySQL</b>
</h5> 

In [50]:
# Code To Connect MySQL
file = open('/Users/katlegomatebane/Documents/Connect/Connect.txt', 'r')
text = file.readlines()

cobi_betika = mysql.connector.connect(host=text[0].strip()
                                      ,database=text[7].strip()
                                      ,user=text[5].strip()
                                      ,password=text[6].strip()
                                      ,port=text[4].strip())

# Connect to MySQL database
try:
    with cobi_betika.cursor() as cursor:
        df = pd.read_sql("SELECT a.playerID\
                                ,a.transDateTime\
                                ,b.msisdn AS phoneNumber\
                                ,left(b.msisdn,3) AS phoneCountry\
                            FROM (SELECT profile_id AS playerID\
                                        ,registration_date AS transDateTime\
                                FROM betika_bi_gh.dim_first_last_gh\
                                WHERE DATE(registration_date) = DATE(CURDATE()- INTERVAL 1 DAY)\
                                ) AS a\
\
                            LEFT JOIN betika_bi_gh.profile AS b\
                            ON a.playerID = b.profile_id\
\
                            LIMIT 1;"
                         ,cobi_betika)
        
finally:
    cobi_betika.close()

DatabaseError: 2005 (HY000): Unknown MySQL server host 'cobi-mysql01.betika.private' (8)

<h5>
    <b> Create File Variables </b>
</h5> 

In [ ]:
report_path = '/Users/katlegomatebane/Documents/UG - EMS Integration/Reports/player Registration/'
name        = '?xml version="1.0" encoding="UTF-8" standalone="yes"?'
date        = df['transDateTime'].min().strftime('%Y%m%d')
counter     = 1
title       = 'RRI_playerRegistration' 

<h5>
    <b> Create Serialisation </b>
</h5> 

In [ ]:
for file in os.listdir(report_path):
        if "xml" in file:
            counter += 1

serial = f"{counter:03}"

In [ ]:
df

,playerID,transDateTime,phoneNumber,phoneCountry
0,3279566,2024-06-13,233539082158,233


<h5>
    <b> Create Data Variables </b>
</h5> 

In [ ]:
df['opener']            = '<?xml version="1.0" encoding="UTF-8" standalone="yes"?>'
df['xmlns:rri']         = 'urn:GSA:RRI:1.0:GSA:rri'
df['xmlns:xsi']         = 'http://www.w3.org/2001/XMLSchema-instance'
df['version']           = '1.0'
df['partionid']         = '1'
df['OperatorId']        = 'Betika'
df['periodType']        = 'RRI_day'
df['reportType']        = 'RRI_playerRegistration'
df['SeqNumber']         = '1'
df['dateTime']          = datetime.now().strftime('%Y-%m-%dT%M:%S.%T')

# PlayerRegistration
df['brandID']           = 'Betika'

# Playerinfo
df['givenNames']        = np.nan
df['familyNames']       = np.nan
df['Nationality']       = np.nan
df['birthDate']         = np.nan
df['genderType']        = np.nan


# PlayerStatusinfo
df['playerStatus']      = 'RRI_active'
df['operatorStatus']    = 'ACTV'

# playerPhoneList
df['phoneType']         = 'RRI_mobile'

#playerIdentification
df['idType']           = np.nan
df['countryCode']      = 'UG'
df['subDivision']      = np.nan
df['idNumber']         = np.nan
df['issueDate']        = np.nan
df['expirationDate']   = np.nan

# playerExclusionList
df['exclusionSource']  = 'RRI_player'
df['startDate']        = np.nan
df['endDate']          = np.nan
df['fromTime']         = np.nan
df['toTime']           = np.nan
df['autoContinuance']  = np.nan
df['removalDateTime']  = np.nan

<h5>
    <b> Prepare Data To Be Hashed By Game ID </b>
</h5> 

In [ ]:
# Create New dataframe that will be used to group data by game id
df_Hash = df

# Place all the row's data into one column
df_Hash['data'] = str(df_Hash).encode()

# convert data column to string 
df_Hash['data'] = df_Hash['data'].astype(str)

# 
df_Hash['data'] = df_Hash.groupby(['transDateTime'])['data'].transform(lambda x: ', '.join(x))

# 
df_Hash = df_Hash.groupby('transDateTime').first().reset_index()

# Keep only the data field & game id
df_Hash = df_Hash[['transDateTime','data']]

In [ ]:
def hash_data(data):
    
    # Encode the transaction data as a string
    data_str = str(data).encode()

    # Create a hash variable and use it to generate a hash of the transaction data
    hashing = hashlib.sha256()
    hashing.update(data_str)
    data_hash = hashing.hexdigest()

    return data_hash

In [ ]:
df_Hash['reportId'] = df_Hash['data'].apply(hash_data)

<h5>
    <b> Map Column Names </b>
</h5> 

In [ ]:
opener                  = df['opener'].astype(str).tolist()
rri                     = df['xmlns:rri'].astype(str).tolist()
xsi                     = df['xmlns:xsi'].astype(str).tolist()
report_id               = df_Hash['reportId'].astype(str).tolist()
Operator_Id             = df['OperatorId'].astype(str).tolist()
period_Start            = df['transDateTime'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()
version                 = df['version'].astype(str).tolist()
partion_id              = df['partionid'].astype(str).tolist()
seq_Number              = df['SeqNumber'].astype(str).tolist()
period_Type             = df['periodType'].astype(str).tolist()
date_Time               = df['dateTime'].astype(str).tolist()
report_Type             = df['reportType'].astype(str).tolist()
brand_Id                = df['brandID'].astype(str).tolist()

# PlayerRegistration
brand_id                = df['brandID'].astype(str).tolist()
trans_Date_Time         = df['transDateTime'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()

# player
player_id               = df['playerID'].astype(str).tolist()

# Playerinfo
given_Names             = df['givenNames'].astype(str).tolist()
family_Names            = df['familyNames'].astype(str).tolist()
Nationality             = df['Nationality'].astype(str).tolist()
birth_Date              = df['birthDate'].astype(str).tolist()
gender_Type             = df['genderType'].astype(str).tolist()
from_Date_Time          = df['transDateTime'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()


# PlayerStatusinfo
player_Status           = df['playerStatus'].astype(str).tolist()
operator_Status         = df['operatorStatus'].astype(str).tolist()
verified_Date_Time      = df['transDateTime'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()

# playerPhoneList
phone_Type              = df['phoneType'].astype(str).tolist()
phone_Country           = df['phoneCountry'].astype(str).tolist()
phone_Number            = df['phoneNumber'].astype(str).tolist()
verified_Date_Time      = df['transDateTime'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()

#playerIdentification
id_Type                 = df['idType'].astype(str).tolist()
country_Code            = df['countryCode'].astype(str).tolist()
sub_Division            = df['subDivision'].astype(str).tolist()
id_Number               = df['idNumber'].astype(str).tolist()
issue_Date              = df['issueDate'].astype(str).tolist()
expiration_Date         = df['expirationDate'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()

# playerExclusionList
exclusion_Source        = df['exclusionSource'].astype(str).tolist()
start_Date              = df['startDate'].astype(str).tolist()
end_Date                = df['endDate'].astype(str).tolist()
from_Time               = df['fromTime'].astype(str).tolist()
to_Time                 = df['toTime'].astype(str).tolist()
auto_Continuance        = df['autoContinuance'].astype(str).tolist()
removal_Date_Time       = df['removalDateTime'].apply(lambda x: x.strftime('%Y-%m-%d')).tolist()


<h5>
    <b> Function To Make XML Readable </b>
</h5> 

In [ ]:
def prettify(element, indent='  '):
    queue = [(0, element)]
    while queue:
        level, element = queue.pop(0)
        children = [(level + 1, child) for child in list(element)]
        if children:
            element.text = '\n' + indent * (level+1)
        if queue: 
            element.tail = '\n' + indent * queue[0][0]
        else:
            element.tail = '\n' + indent * (level-1) 
        queue[0:0] = children

<h5>
    <b> Create XML File </b>
</h5> 

In [ ]:
xml_doc = ET.Element(name)
gaming_Report = ET.SubElement(xml_doc, 'rri:gamingReport'
                            ,xmlnsrri=rri[0]
                            ,xmlnsxsi=xsi[0]
                            ,version=version[0]
                            ,operatiorId=Operator_Id[0]
                            ,partionId=partion_id[0]
                            ,periodType=period_Type[0]
                            ,periodStart=period_Start[0]
                            ,reportType=report_Type[0]
                            ,reportid=report_id[0]
                            ,seqNumber=seq_Number[0]
                            ,dateTime=date_Time[0])
playerRegistration = ET.SubElement(gaming_Report, 'playerRegistration' 
                             ,brandId=brand_Id[0]
                             ,transDateTime=trans_Date_Time[0])
ET.SubElement(playerRegistration,'player'
              ,playerId=player_id[0])
ET.SubElement(playerRegistration,'playerInfo'
             ,givenNames=given_Names[0]
             ,familyNames=family_Names[0]
             ,nationality=Nationality[0]
             ,birthDate=birth_Date[0]
             ,genderType=gender_Type[0]
             ,fromDateTime=date_Time[0])
playerStatusInfo = ET.SubElement(playerRegistration, 'playerStatusInfo' 
                                ,playerStatus=player_Status[0]
                                ,operatorStatus=operator_Status[0]
                                ,fromDateTime=from_Date_Time[0]
                                ,verifiedDateTime=verified_Date_Time[0])
playerPhoneList = ET.SubElement(playerRegistration, 'playerPhoneList')
ET.SubElement(playerPhoneList,'playerPhone'
             ,phoneType=phone_Type[0]
             ,phoneCountry=phone_Country[0]
             ,phoneNumber=phone_Number[0]
             ,verifiedDateTime=verified_Date_Time[0])
playerIdentificationList = ET.SubElement(playerRegistration, 'playerIdentificationList')
ET.SubElement(playerIdentificationList,'playerIdentification'
             ,idType=id_Type[0]
             ,countryCode=country_Code[0]
             ,subDivision=sub_Division[0]
             ,idNumber=id_Number[0]
             ,issueDate=issue_Date[0]
             ,expirationDate=expiration_Date[0])
playerExclusion = ET.SubElement(playerRegistration, 'playerExclusionList')
ET.SubElement(playerExclusion,'playerExclusion'
             ,exclusionSource=exclusion_Source[0]
             ,startDate=start_Date[0]
             ,endDate=end_Date[0]
             ,fromTime=from_Time[0]
             ,toTime=to_Time[0]
             ,autoContinuance=auto_Continuance[0]
             ,removalDateTime=removal_Date_Time[0])

prettify(xml_doc)

<h5>
    <b> Export XML File To Folder </b>
</h5> 

In [ ]:
tree = ET.ElementTree(xml_doc)
tree.write(report_path+title+'_'+serial+'_'+date+'.xml',encoding="utf-8")